In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
data = pd.read_csv('data.csv')

#pre-processing to calculate outside good shares
shares = data[['Market_ID','Inside Good Share','PPO']].copy()
shares['PPO Share'] = data['Inside Good Share'] * data['PPO']
shares['HMO Share'] = data['Inside Good Share'] * (1 - data['PPO'])

group_shares = shares.groupby('Market_ID').sum()
group_shares['Outside Good Share'] = 1 - group_shares['Inside Good Share']

data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')

print data.min()

Market_ID             1.000000
Plan_ID               1.000000
PPO                   0.000000
Network Score         0.780000
Satisfaction Score    0.725000
Premium               2.269728
Inside Good Share     0.037670
Outside Good Share    0.096070
dtype: float64


In [3]:
#first estimate using logit
class logit(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(logit, self).__init__(*args, **kwds)

        
    def momcond(self, params):
        #unwrap stuff
        shares = np.array(self.endog).transpose()
        exog = np.array(self.exog)
        instr = np.array(self.instrument)
        
        lshare = np.log(shares[0]) -  np.log(shares[1])
        lshare = lshare.transpose()
       
        lshare_fit = np.matmul(exog,params) #linear equation    
        
        xi = lshare_fit - lshare
        g = instr * xi[:, np.newaxis]
        
        return g

In [4]:
#calculate hausmann insturments
mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
hausman_instr = plan_dum

#set up x and y
y = data[['Inside Good Share','Outside Good Share']]
x =  data[['Network Score','Satisfaction Score','PPO','Premium']]

In [5]:
#set up initial est
beta_init = np.full(len(x.columns),1)

#set up model
model = logit(y , x, hausman_instr)

result = model.fit(beta_init, maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 292
         Function evaluations: 497
Optimization terminated successfully.
         Current function value: 0.005149
         Iterations: 139
         Function evaluations: 249
                                             logit Results                                             
Dep. Variable:     ['Inside Good Share', 'Outside Good Share']   Hansen J:                        16.99
Model:                                                   logit   Prob (Hansen J):                 0.108
Method:                                                    GMM                                         
Date:                                         Wed, 10 Oct 2018                                         
Time:                                                 09:59:56                                         
No. Observations:                                         3300                                  

In [72]:
#correct way to compute shares...

#compute exp(delta_j)
delta = data.copy()[['Plan_ID','Market_ID','PPO','Premium']]
x = np.array(x)
beta = result.params
alpha = abs(beta[3])
delta['exp_delta'] = np.exp(np.matmul(x,beta))

#compute 1 + sum_j exp(delta_j)
sum_delta = delta.groupby('Market_ID').sum()
sum_delta['sum_exp_delta'] = 1 + sum_delta['exp_delta'] 

delta = pd.merge(delta, sum_delta[['sum_exp_delta']], 
                right_index=True, left_on = 'Market_ID')

#compute s_j
delta['fitted_share'] = delta['exp_delta']/delta['sum_exp_delta']

print delta.groupby('Market_ID').sum().mean()

print delta['fitted_share'].max(), delta['fitted_share'].min(), delta['fitted_share'].mean(), np.sqrt(delta['fitted_share'].var())

Plan_ID          47.138333
PPO               2.720000
Premium          13.562795
exp_delta         6.297526
sum_exp_delta    44.228315
fitted_share      0.843098
dtype: float64
0.3211705667781474 0.04175067681732296 0.153290506773 0.0527060652944428


In [73]:
#simplified way to calculate shares

def comp_shares(x,p,alpha,beta):
    beta = beta[:-1]
    delta_js = np.matmul(x,beta) - alpha*p
    numer = np.exp(delta_js)
    denom = 1+sum(np.exp(delta_js))
    return numer/denom

#parameters
alpha = abs(result.params[3])

#data
xes = np.array( data[['Plan_ID','Network Score','Satisfaction Score','PPO']].groupby('Plan_ID').mean() )
prices = np.array( data[['Plan_ID','Premium']].groupby('Plan_ID').mean() ).squeeze()

#calc
shares = comp_shares(xes, prices,alpha,beta)

print shares

[0.07777855 0.04112888 0.08286704 0.0605475  0.04363872 0.05471835
 0.07445872 0.06017672 0.06119211 0.03388444 0.04466438 0.07627382
 0.05354786 0.05155689 0.08099884 0.05066723]


In [74]:
#agregate elasticities

#shares = comp_shares(xes, prices,alpha,beta)
shares = np.array(delta[['Plan_ID','fitted_share']].groupby('Plan_ID').mean()).squeeze()


#set up matrix skeleton
own_price = np.identity(len(shares))
cross_price = 1 - own_price

#actually calculate elasticity
cross_elasticity = shares * alpha * prices
own_elasticity  = -(1-shares) * alpha * prices

elasticity =  cross_price*cross_elasticity + own_price *own_elasticity

print elasticity[1][1],elasticity[2][1],elasticity[3][1] #second index tells you wrt what good i.e. denom

print elasticity

-4.308596601690402 0.5554552982100271 0.5554552982100271
[[-4.32655865  0.5554553   1.08916071  0.82684009  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942 -4.3085966   1.08916071  0.82684009  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942  0.5554553  -4.2870623   0.82684009  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942  0.5554553   1.08916071 -4.37731375  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942  0.5554553   1.08916071  0.82684009 -4.25288515  0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945

In [75]:
#solve for marginal costs
shares_vector = np.array([shares])

#actually calculate elasticity
cross_deriv = alpha * np.matmul(shares_vector.transpose() , shares_vector )
own_deriv  = - alpha * (1-shares)  * shares
derivative =  cross_price*cross_deriv + own_price *own_deriv

print derivative[1][2], derivative[2][1]
print shares[1]*shares[2]*alpha, derivative[1][2]

#take inverse and calc markup
inv_derivative = np.linalg.inv(derivative)

markup = np.matmul(inv_derivative, shares_vector.transpose()) 

#get MC
mc = prices - markup.transpose()[0]

print markup.transpose()[0]
print mc

0.04769156636161652 0.04769156636161652
0.04769156636161652 0.04769156636161652
[0.33252165 0.33252165 0.33252165 0.33252165 0.33252165 0.33252165
 0.33252165 0.33252165 0.33252165 0.33252165 0.33252165 0.33252165
 0.33252165 0.33252165 0.33252165 0.33252165]
[2.25894503 2.0269905  2.27544055 2.19197119 2.00760625 2.02311588
 2.26307034 2.0775342  2.19892493 2.00506537 1.99193096 2.26812745
 2.18890878 2.0783699  2.26398501 2.03350633]


In [76]:
def profits(p, rebate=.25, x=xes, beta=beta):
    #p[p < 0] = 0 #set things if necessary?
    s_js = comp_shares(x, p, alpha, beta)
    return - np.matmul(s_js, np.array([prices-mc]).transpose())

print profits(prices)

[-0.31526378]


In [77]:
import numpy as np
from scipy.optimize import minimize

res = minimize(profits, prices, method='nelder-mead',
     options={'xtol': 1e-8, 'disp': True})

print(res.x) #doesn't seem right

Optimization terminated successfully.
         Current function value: -0.332522
         Iterations: 261
         Function evaluations: 787
[-19.43519922  -7.01400183  -2.88970246  -2.21091796  16.45317146
   7.59678455  -1.55736601   8.36333496   5.02757004   6.82512629
   5.52850426  -0.26647718   7.98034276   8.90988572  -3.03417415
   5.12151029]


In [ ]:
#profits per enrollee
#outside good shares
#consumer surplus

Plan_ID          47.138333
PPO               2.720000
Premium          13.562795
exp_delta         6.297526
sum_exp_delta    44.228315
fitted_share      0.843098
dtype: float64
0.3211705667781474 0.04175067681732296 0.153290506773 0.0527060652944428
